In [ ]:
import random
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import cv2
import os
from os import listdir
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import keras
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path="/content/drive/MyDrive/assignment/COVID_19_CT_Data"

In [ ]:
ids=[]
classify=[]
for dir in listdir(os.path.join(path,'Train')):
  for id in listdir(os.path.join(path,'Train',dir)):
    ids.append(id)
    classify.append(dir) 

In [ ]:
train = pd.DataFrame(columns=['imageid','label'])

In [ ]:
train.imageid=ids
train.label=classify
train.head()

,imageid,label
0,Non-Covid (1009).png,non-COVID
1,Non-Covid (1021).png,non-COVID
2,Non-Covid (1018).png,non-COVID
3,Non-Covid (1005).png,non-COVID
4,Non-Covid (1001).png,non-COVID


In [ ]:
train.label.value_counts()

COVID        700
non-COVID    700
Name: label, dtype: int64

In [ ]:
image_size = [] #list of tuples containing shape(height, width, channel) of images
for dir in listdir(os.path.join(path,'Train')):
  for id in tqdm(listdir(os.path.join(path,'Train', dir))):
    img = cv2.imread(os.path.join(path,'Train', dir, id))
    h, w, c = img.shape

    image_size.append((h, w, c))

In [ ]:
# print unique values in image_size list
set(image_size)

{(100, 100, 3)}

In [ ]:
image_size = [] #list of tuples containing shape(height, width, channel) of images
for dir in listdir(os.path.join(path,'Test')):
  for id in tqdm(listdir(os.path.join(path,'Test', dir))):
    img = cv2.imread(os.path.join(path,'Test', dir, id))
    h, w, c = img.shape

    image_size.append((h, w, c))

In [ ]:
# print unique values in image_size list
set(image_size)

{(100, 100, 3)}

**All images are of same size.**

In [ ]:
import shutil
for dir in listdir(os.path.join(path,'Train')):
    for id in tqdm(listdir(os.path.join(path,'Train',dir))):
        shutil.copy(os.path.join(path, 'Train', dir, id), 'Covid_Data/Train/'+id)
    

In [ ]:
import shutil
for dir in listdir(os.path.join(path,'Test')):
    for id in tqdm(listdir(os.path.join(path,'Test',dir))):
        shutil.copy(os.path.join(path, 'Test', dir, id), 'Covid_Data/Test/'+id)

In [ ]:
test_ids = []
test_classify = []
for id in tqdm(listdir('Covid_Data/Test/')):
    test_ids.append(id)
    if id.startswith('Covid'):
        test_classify.append(0) # This is how the data generator perceives COVID - 0, non-COVID - 1
    elif id.startswith('Non-Covid'):
        test_classify.append(1)


In [ ]:
test_data = pd.DataFrame({'imageid': test_ids, 'label': test_classify})

In [ ]:
from sklearn.model_selection import train_test_split
#splitting the data into train & cv
train_data, val_data = train_test_split(train, test_size=0.15, random_state=42,stratify=train['label'])
print(train_data.shape)
print(val_data.shape)

(1190, 2)
(210, 2)


In [ ]:
train_ids = train_data.imageid
val_ids = val_data.imageid
train_classify = train_data.label
val_classify = train_data.label

In [ ]:
for id in tqdm(val_ids):
  shutil.move('Covid_Data/Train/'+id, 'Covid_Data/Val/'+id)


In [ ]:
train_image_gen = ImageDataGenerator(rescale=1./255)
train_batches = train_image_gen.flow_from_dataframe(train_data, x_col='imageid', y_col="label", class_mode="binary",
                                                   directory = 'Covid_Data/Train', batch_size = 32, target_size=(100, 100))
val_batches = train_image_gen.flow_from_dataframe(val_data, x_col='imageid', y_col="label", class_mode="binary",
                                                   directory = 'Covid_Data/Val', batch_size = 32, target_size=(100, 100))

Found 1190 validated image filenames belonging to 2 classes.
Found 210 validated image filenames belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([

    # Conv 1
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),

    # Conv 2
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),

    
    # Conv 3
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Dropout(0.5),
    
    # Conv 4
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

In [ ]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 49, 49, 16)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 23, 23, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 21, 21, 64)      

In [ ]:
# compile model
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])



In [ ]:
model.fit(train_batches, epochs=30, validation_data=val_batches)

Epoch 1/30
38/38 [==============================] - 2s 34ms/step - loss: 0.9330 - accuracy: 0.5190 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/30
38/38 [==============================] - 1s 29ms/step - loss: 0.6814 - accuracy: 0.5548 - val_loss: 0.6899 - val_accuracy: 0.5000
Epoch 3/30
38/38 [==============================] - 1s 29ms/step - loss: 0.6337 - accuracy: 0.6414 - val_loss: 0.6893 - val_accuracy: 0.5000
Epoch 4/30
38/38 [==============================] - 1s 30ms/step - loss: 0.6364 - accuracy: 0.6160 - val_loss: 0.6863 - val_accuracy: 0.5000
Epoch 5/30
38/38 [==============================] - 1s 29ms/step - loss: 0.5797 - accuracy: 0.6993 - val_loss: 0.6958 - val_accuracy: 0.5000
Epoch 6/30
38/38 [==============================] - 1s 29ms/step - loss: 0.5307 - accuracy: 0.7221 - val_loss: 0.6850 - val_accuracy: 0.5000
Epoch 7/30
38/38 [==============================] - 1s 29ms/step - loss: 0.5094 - accuracy: 0.7596 - val_loss: 0.6898 - val_accuracy: 0.5000
Epoch 8/30
38

In [ ]:
model.evaluate(val_batches)

7/7 [==============================] - 0s 20ms/step - loss: 0.5188 - accuracy: 0.8476


[0.5187916159629822, 0.8476190567016602]

In [ ]:
predictions = []
for id in tqdm(test_data.imageid.values):

    img = cv2.imread('/content/Covid_Data/Test/' + id)
    X = np.empty((1,100,100,3),dtype=np.uint8)
    X[0,] = img
    predictions.append(int(model.predict(X)[0][0]))

In [ ]:
confusion_matrix(test_classify, predictions)

array([[302,   0],
       [283,   1]])

In [ ]:
report = classification_report(test_classify, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.52      1.00      0.68       302
           1       1.00      0.00      0.01       284

    accuracy                           0.52       586
   macro avg       0.76      0.50      0.34       586
weighted avg       0.75      0.52      0.35       586

